<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/Visualization/NB_02_Pareto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
2+3

5

In [3]:
import pandas as pd
from plotly.graph_objects import Figure, Scatter, Bar

In [4]:
data = {
    "xml_parent_city_name": ["New York"] * 11,
    "xml_city_name": [
        "New York", "Brooklyn", "Hoboken", "Manhattan", "Union City",
        "Jersey City", "Queens", "West New York", "Newark", "Elizabeth", "Paterson"
    ],
    "distance_from_parent": [0.00, 3.31, 6.44, 6.52, 7.55, 7.65, 7.68, 7.70, 13.13, 14.08, 19.64],
    "leads": [1087, 89, 25, 73, 15, 61, 135, 23, 6, 11, 10],
    "percentage": [70.81, 5.80, 1.63, 4.76, 0.98, 3.97, 8.79, 1.50, 0.39, 0.72, 0.65],
    "cumulative_percentage": [70.81, 76.61, 78.24, 83.00, 83.98, 87.95, 96.74, 98.24, 98.63, 99.35, 100.00],
}

df = pd.DataFrame(data)

df_plot = df
df_plot

,xml_parent_city_name,xml_city_name,distance_from_parent,leads,percentage,cumulative_percentage
0,New York,New York,0.00,1087,70.81,70.81
1,New York,Brooklyn,3.31,89,5.80,76.61
2,New York,Hoboken,6.44,25,1.63,78.24
3,New York,Manhattan,6.52,73,4.76,83.00
4,New York,Union City,7.55,15,0.98,83.98
5,New York,Jersey City,7.65,61,3.97,87.95
6,New York,Queens,7.68,135,8.79,96.74
7,New York,West New York,7.70,23,1.50,98.24
8,New York,Newark,13.13,6,0.39,98.63
9,New York,Elizabeth,14.08,11,0.72,99.35


In [10]:
df_plot_02 = df_plot.copy()
df_plot_02 = df_plot_02.sort_values(by="leads", ascending=False)
df_plot_02["cumulative_percentage"] = df_plot_02["percentage"].cumsum().round(2)
# df_plot_02 = df_plot_02.reset_index(drop=True)
df_plot_02

,xml_parent_city_name,xml_city_name,distance_from_parent,leads,percentage,cumulative_percentage
0,New York,New York,0.00,1087,70.81,70.81
6,New York,Queens,7.68,135,8.79,79.60
1,New York,Brooklyn,3.31,89,5.80,85.40
3,New York,Manhattan,6.52,73,4.76,90.16
5,New York,Jersey City,7.65,61,3.97,94.13
2,New York,Hoboken,6.44,25,1.63,95.76
7,New York,West New York,7.70,23,1.50,97.26
4,New York,Union City,7.55,15,0.98,98.24
9,New York,Elizabeth,14.08,11,0.72,98.96
10,New York,Paterson,19.64,10,0.65,99.61


In [6]:
def graph_pareto(dataframe, value_col, percentage_col, cumulative_col, x_col, extra_col):
    # Copiar y ordenar los datos
    df = dataframe.copy()

    # Identificar la primera barra donde el porcentaje acumulado >= 80
    first_above_80 = df[df[cumulative_col] >= 80].index.min()

    # Determinar los colores para las barras basados en el porcentaje acumulado
    bar_colors = [
        'rgb(71, 71, 135)' if idx <= first_above_80 else 'rgb(112, 111, 211)'
        for idx in df.index
    ]

    # Determinar los colores para la línea y los puntos de porcentaje acumulado (rojo oscuro a rojo más claro)
    line_and_marker_colors = [
        'rgb(192, 57, 43)' if idx <= first_above_80 else 'rgb(255, 165, 123)'
        for idx in df.index
    ]

    # Crear etiquetas de texto para las barras con la nueva columna
    bar_hovertext = [
        f"{x_label}: {x_val}<br>Count: {value}<br>Percentage: {percentage:.2f}%<br>{extra_col}: {extra_val}"
        for x_label, x_val, value, percentage, extra_val in zip(
            df[x_col], df[x_col], df[value_col], df[percentage_col], df[extra_col]
        )
    ]

    data = [
        Bar(
            name="Count",
            x=df[x_col],
            y=df[value_col],
            marker={"color": bar_colors},
            hovertext=bar_hovertext,
            hoverinfo="text",
            text=[
                f"{value} ({percentage:.2f}%)"
                for value, percentage in zip(df[value_col], df[percentage_col])
            ],
            textposition="outside",
            textfont={"size": 10},
        ),
        Scatter(
            name="Percentage",
            x=df[x_col],
            y=df[cumulative_col],
            yaxis="y2",
            mode='lines+markers',
            line={
                "color": line_and_marker_colors[0],
                "width": 3
            },
            marker=dict(
                color=line_and_marker_colors,
                size=8,
                line=dict(width=1, color='rgb(0,0,0)')
            ),
            hovertext=[
                f"{x_label}: {x_val}<br>Cumulative Percentage: {cum_percentage:.2f}%<br>{extra_col}: {extra_val}"
                for x_label, x_val, cum_percentage, extra_val in zip(
                    df[x_col], df[x_col], df[cumulative_col], df[extra_col]
                )
            ],
            hoverinfo="text",
        ),
    ]

    layout = {
        "title": {
            'text': f"Pareto Analysis",
            'font': dict(size=30)
        },
        "font": {
            "size": 14,
            "color": "rgb(44, 44, 84)",
            "family": "Times New Roman, monospace"
        },
        "margin": {
            "b": 50,
            "l": 50,
            "r": 50,
            "t": 50,
        },
        "height": 400,
        "plot_bgcolor": "rgb(255, 255, 255)",
        "legend": {
            "x": 0.79,
            "y": 1.2,
            "font": {
                "size": 12,
                "color": "rgb(44, 44, 84)",
                "family": "Courier New, monospace"
            },
            'orientation': 'h',
        },
        "yaxis": {
            "title": f"Count {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
        },
        "yaxis2": {
            "side": "right",
            "range": [0, 100],
            "title": f"Percentage {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
            "overlaying": "y",
            "ticksuffix": " %",
        },
    }

    # Crear el gráfico
    fig = Figure(data=data, layout=layout)
    fig.show()

In [13]:
graph_pareto(
    dataframe=df_plot,
    value_col="leads",
    percentage_col="percentage",
    cumulative_col="cumulative_percentage",
    x_col="xml_city_name",
    extra_col="distance_from_parent"
)

In [11]:
def graph_pareto(dataframe, value_col, percentage_col, cumulative_col, x_col, extra_col):
    # Copiar y ordenar los datos
    df = dataframe.copy()
    df = df.reset_index(drop=True)

    # Identificar la primera barra donde el porcentaje acumulado >= 80
    first_above_80 = df[df[cumulative_col] >= 80].index.min()

    # Determinar los colores para las barras basados en el porcentaje acumulado
    bar_colors = [
        'rgb(71, 71, 135)' if idx <= first_above_80 else 'rgb(112, 111, 211)'
        for idx in df.index
    ]

    # Determinar los colores para la línea y los puntos de porcentaje acumulado (rojo oscuro a rojo más claro)
    line_and_marker_colors = [
        'rgb(192, 57, 43)' if idx <= first_above_80 else 'rgb(255, 165, 123)'
        for idx in df.index
    ]

    # Crear etiquetas de texto para las barras con la nueva columna
    bar_hovertext = [
        f"{x_label}: {x_val}<br>Count: {value}<br>Percentage: {percentage:.2f}%<br>{extra_col}: {extra_val}"
        for x_label, x_val, value, percentage, extra_val in zip(
            df[x_col], df[x_col], df[value_col], df[percentage_col], df[extra_col]
        )
    ]

    data = [
        Bar(
            name="Count",
            x=df[x_col],
            y=df[value_col],
            marker={"color": bar_colors},
            hovertext=bar_hovertext,
            hoverinfo="text",
            text=[
                f"{value} ({percentage:.2f}%)"
                for value, percentage in zip(df[value_col], df[percentage_col])
            ],
            textposition="outside",
            textfont={"size": 10},
        ),
        Scatter(
            name="Percentage",
            x=df[x_col],
            y=df[cumulative_col],
            yaxis="y2",
            mode='lines+markers',
            line={
                "color": line_and_marker_colors[0],
                "width": 3
            },
            marker=dict(
                color=line_and_marker_colors,
                size=8,
                line=dict(width=1, color='rgb(0,0,0)')
            ),
            hovertext=[
                f"{x_label}: {x_val}<br>Cumulative Percentage: {cum_percentage:.2f}%<br>{extra_col}: {extra_val}"
                for x_label, x_val, cum_percentage, extra_val in zip(
                    df[x_col], df[x_col], df[cumulative_col], df[extra_col]
                )
            ],
            hoverinfo="text",
        ),
    ]

    layout = {
        "title": {
            'text': f"Pareto Analysis",
            'font': dict(size=30)
        },
        "font": {
            "size": 14,
            "color": "rgb(44, 44, 84)",
            "family": "Times New Roman, monospace"
        },
        "margin": {
            "b": 50,
            "l": 50,
            "r": 50,
            "t": 50,
        },
        "height": 400,
        "plot_bgcolor": "rgb(255, 255, 255)",
        "legend": {
            "x": 0.79,
            "y": 1.2,
            "font": {
                "size": 12,
                "color": "rgb(44, 44, 84)",
                "family": "Courier New, monospace"
            },
            'orientation': 'h',
        },
        "yaxis": {
            "title": f"Count {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
        },
        "yaxis2": {
            "side": "right",
            "range": [0, 100],
            "title": f"Percentage {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
            "overlaying": "y",
            "ticksuffix": " %",
        },
    }

    # Crear el gráfico
    fig = Figure(data=data, layout=layout)
    fig.show()

In [10]:
df_plot_02


,xml_parent_city_name,xml_city_name,distance_from_parent,leads,percentage,cumulative_percentage
0,New York,New York,0.00,1087,70.81,70.81
1,New York,Queens,7.68,135,8.79,79.60
2,New York,Brooklyn,3.31,89,5.80,85.40
3,New York,Manhattan,6.52,73,4.76,90.16
4,New York,Jersey City,7.65,61,3.97,94.13
5,New York,Hoboken,6.44,25,1.63,95.76
6,New York,West New York,7.70,23,1.50,97.26
7,New York,Union City,7.55,15,0.98,98.24
8,New York,Elizabeth,14.08,11,0.72,98.96
9,New York,Paterson,19.64,10,0.65,99.61


In [12]:
graph_pareto(
    dataframe=df_plot_02,
    value_col="leads",
    percentage_col="percentage",
    cumulative_col="cumulative_percentage",
    x_col="xml_city_name",
    extra_col="distance_from_parent"
)

In [14]:
def graph_pareto(dataframe, value_col, percentage_col, cumulative_col, x_col, extra_cols):
    # Copiar y ordenar los datos
    df = dataframe.copy()
    df = df.reset_index(drop=True)

    # Identificar la primera barra donde el porcentaje acumulado >= 80
    first_above_80 = df[df[cumulative_col] >= 80].index.min()

    # Determinar los colores para las barras basados en el porcentaje acumulado
    bar_colors = [
        'rgb(71, 71, 135)' if idx <= first_above_80 else 'rgb(112, 111, 211)'
        for idx in df.index
    ]

    # Determinar los colores para la línea y los puntos de porcentaje acumulado (rojo oscuro a rojo más claro)
    line_and_marker_colors = [
        'rgb(192, 57, 43)' if idx <= first_above_80 else 'rgb(255, 165, 123)'
        for idx in df.index
    ]

    # Crear etiquetas de texto para las barras con múltiples columnas extra
    def create_hovertext(row):
        extra_info = "<br>".join([f"{col}: {row[col]}" for col in extra_cols])
        return (
            f"{x_col}: {row[x_col]}<br>Count: {row[value_col]}<br>"
            f"Percentage: {row[percentage_col]:.2f}%<br>{extra_info}"
        )

    bar_hovertext = [create_hovertext(row) for _, row in df.iterrows()]

    # Crear etiquetas para la línea y los puntos
    line_hovertext = [
        create_hovertext(row).replace(
            f"Percentage: {row[percentage_col]:.2f}%",
            f"Cumulative Percentage: {row[cumulative_col]:.2f}%"
        )
        for _, row in df.iterrows()
    ]

    data = [
        Bar(
            name="Count",
            x=df[x_col],
            y=df[value_col],
            marker={"color": bar_colors},
            hovertext=bar_hovertext,
            hoverinfo="text",
            text=[
                f"{value} ({percentage:.2f}%)"
                for value, percentage in zip(df[value_col], df[percentage_col])
            ],
            textposition="outside",
            textfont={"size": 10},
        ),
        Scatter(
            name="Percentage",
            x=df[x_col],
            y=df[cumulative_col],
            yaxis="y2",
            mode='lines+markers',
            line={
                "color": line_and_marker_colors[0],
                "width": 3
            },
            marker=dict(
                color=line_and_marker_colors,
                size=8,
                line=dict(width=1, color='rgb(0,0,0)')
            ),
            hovertext=line_hovertext,
            hoverinfo="text",
        ),
    ]

    layout = {
        "title": {
            'text': f"Pareto Analysis",
            'font': dict(size=30)
        },
        "font": {
            "size": 14,
            "color": "rgb(44, 44, 84)",
            "family": "Times New Roman, monospace"
        },
        "margin": {
            "b": 50,
            "l": 50,
            "r": 50,
            "t": 50,
        },
        "height": 400,
        "plot_bgcolor": "rgb(255, 255, 255)",
        "legend": {
            "x": 0.79,
            "y": 1.2,
            "font": {
                "size": 12,
                "color": "rgb(44, 44, 84)",
                "family": "Courier New, monospace"
            },
            'orientation': 'h',
        },
        "yaxis": {
            "title": f"Count {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
        },
        "yaxis2": {
            "side": "right",
            "range": [0, 100],
            "title": f"Percentage {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
            "overlaying": "y",
            "ticksuffix": " %",
        },
    }

    # Crear el gráfico
    fig = Figure(data=data, layout=layout)
    fig.show()


In [15]:
df_plot_02

,xml_parent_city_name,xml_city_name,distance_from_parent,leads,percentage,cumulative_percentage
0,New York,New York,0.00,1087,70.81,70.81
6,New York,Queens,7.68,135,8.79,79.60
1,New York,Brooklyn,3.31,89,5.80,85.40
3,New York,Manhattan,6.52,73,4.76,90.16
5,New York,Jersey City,7.65,61,3.97,94.13
2,New York,Hoboken,6.44,25,1.63,95.76
7,New York,West New York,7.70,23,1.50,97.26
4,New York,Union City,7.55,15,0.98,98.24
9,New York,Elizabeth,14.08,11,0.72,98.96
10,New York,Paterson,19.64,10,0.65,99.61


In [26]:
list(df_plot_02.columns)

['xml_parent_city_name',
 'xml_city_name',
 'distance_from_parent',
 'leads',
 'percentage',
 'cumulative_percentage']

In [29]:
graph_pareto(
  dataframe=df_plot_02,
  value_col="leads",
  percentage_col="percentage",
  cumulative_col="cumulative_percentage",
  x_col="xml_city_name",
  extra_cols=list(df_plot_02.columns)
)

In [28]:
def graph_pareto(dataframe, value_col, percentage_col, cumulative_col, x_col, extra_cols):
    # Copiar y ordenar los datos
    df = dataframe.copy()
    df = df.reset_index(drop=True)

    # Identificar la primera barra donde el porcentaje acumulado >= 80
    first_above_80 = df[df[cumulative_col] >= 80].index.min()

    # Determinar los colores para las barras basados en el porcentaje acumulado
    bar_colors = [
        'rgb(71, 71, 135)' if idx <= first_above_80 else 'rgb(112, 111, 211)'
        for idx in df.index
    ]

    # Determinar los colores para la línea y los puntos de porcentaje acumulado (rojo oscuro a rojo más claro)
    line_and_marker_colors = [
        'rgb(192, 57, 43)' if idx <= first_above_80 else 'rgb(255, 165, 123)'
        for idx in df.index
    ]

    # Crear etiquetas de texto únicamente con las columnas en extra_cols
    def create_hovertext(row):
        return "<br>".join([f"{col}: {row[col]}" for col in extra_cols])

    # Hovertext para las barras
    bar_hovertext = [create_hovertext(row) for _, row in df.iterrows()]

    # Hovertext para la línea de porcentaje acumulado
    line_hovertext = [create_hovertext(row) for _, row in df.iterrows()]

    # Texto sobre las barras (manteniendo Count y Percentage)
    bar_text = [
        f"{value} ({percentage:.2f}%)"
        for value, percentage in zip(df[value_col], df[percentage_col])
    ]

    data = [
        Bar(
            name="Count",
            x=df[x_col],
            y=df[value_col],
            marker={"color": bar_colors},
            hovertext=bar_hovertext,
            hoverinfo="text",
            text=bar_text,
            textposition="outside",
            textfont={"size": 10},
        ),
        Scatter(
            name="Percentage",
            x=df[x_col],
            y=df[cumulative_col],
            yaxis="y2",
            mode='lines+markers',
            line={
                "color": line_and_marker_colors[0],
                "width": 3
            },
            marker=dict(
                color=line_and_marker_colors,
                size=8,
                line=dict(width=1, color='rgb(0,0,0)')
            ),
            hovertext=line_hovertext,
            hoverinfo="text",
        ),
    ]

    layout = {
        "title": {
            'text': f"Pareto Analysis",
            'font': dict(size=30)
        },
        "font": {
            "size": 14,
            "color": "rgb(44, 44, 84)",
            "family": "Times New Roman, monospace"
        },
        "margin": {
            "b": 50,
            "l": 50,
            "r": 50,
            "t": 50,
        },
        "height": 400,
        "plot_bgcolor": "rgb(255, 255, 255)",
        "legend": {
            "x": 0.79,
            "y": 1.2,
            "font": {
                "size": 12,
                "color": "rgb(44, 44, 84)",
                "family": "Courier New, monospace"
            },
            'orientation': 'h',
        },
        "yaxis": {
            "title": f"Count {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
        },
        "yaxis2": {
            "side": "right",
            "range": [0, 100],
            "title": f"Percentage {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
            "overlaying": "y",
            "ticksuffix": " %",
        },
    }

    # Crear el gráfico
    fig = Figure(data=data, layout=layout)
    fig.show()


In [22]:
def graph_pareto(dataframe, value_col, percentage_col, cumulative_col, x_col, extra_cols):
    # Copiar y ordenar los datos
    df = dataframe.copy()
    df = df.reset_index(drop=True)

    # Identificar la primera barra donde el porcentaje acumulado >= 80
    first_above_80 = df[df[cumulative_col] >= 80].index.min()

    # Determinar los colores para las barras basados en el porcentaje acumulado
    bar_colors = [
        'rgb(71, 71, 135)' if idx <= first_above_80 else 'rgb(112, 111, 211)'
        for idx in df.index
    ]

    # Determinar los colores para la línea y los puntos de porcentaje acumulado (rojo oscuro a rojo más claro)
    line_and_marker_colors = [
        'rgb(192, 57, 43)' if idx <= first_above_80 else 'rgb(255, 165, 123)'
        for idx in df.index
    ]

    # Crear etiquetas de texto únicamente con las columnas en extra_cols
    def create_hovertext(row):
        return "<br>".join([f"{col}: {row[col]}" for col in extra_cols])

    bar_hovertext = [create_hovertext(row) for _, row in df.iterrows()]
    line_hovertext = [create_hovertext(row) for _, row in df.iterrows()]

    data = [
        Bar(
            name="Count",
            x=df[x_col],
            y=df[value_col],
            marker={"color": bar_colors},
            hovertext=bar_hovertext,
            hoverinfo="text",
        ),
        Scatter(
            name="Percentage",
            x=df[x_col],
            y=df[cumulative_col],
            yaxis="y2",
            mode='lines+markers',
            line={
                "color": line_and_marker_colors[0],
                "width": 3
            },
            marker=dict(
                color=line_and_marker_colors,
                size=8,
                line=dict(width=1, color='rgb(0,0,0)')
            ),
            hovertext=line_hovertext,
            hoverinfo="text",
        ),
    ]

    layout = {
        "title": {
            'text': f"Pareto Analysis",
            'font': dict(size=30)
        },
        "font": {
            "size": 14,
            "color": "rgb(44, 44, 84)",
            "family": "Times New Roman, monospace"
        },
        "margin": {
            "b": 50,
            "l": 50,
            "r": 50,
            "t": 50,
        },
        "height": 400,
        "plot_bgcolor": "rgb(255, 255, 255)",
        "legend": {
            "x": 0.79,
            "y": 1.2,
            "font": {
                "size": 12,
                "color": "rgb(44, 44, 84)",
                "family": "Courier New, monospace"
            },
            'orientation': 'h',
        },
        "yaxis": {
            "title": f"Count {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
        },
        "yaxis2": {
            "side": "right",
            "range": [0, 100],
            "title": f"Percentage {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
            "overlaying": "y",
            "ticksuffix": " %",
        },
    }

    # Crear el gráfico
    fig = Figure(data=data, layout=layout)
    fig.show()
